In [133]:
# Import necessary libraries.
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import kstest
import numpy as np
import scipy.stats as spstats
from scipy.stats import shapiro
from sklearn.preprocessing import *
import warnings
from sklearn import linear_model
from skater.core.explanations import Interpretation
from skater.model import InMemoryModel
from skater.core.local_interpretation.lime.lime_tabular import LimeTabularExplainer
from IPython.display import display, HTML, clear_output

from progressbar import *

import data_clean
import feature_engineering
import feature_filtering
import step_by_step
import evaluation

reload(data_clean)
reload(feature_engineering)
plt.style.use('ggplot')
%matplotlib inline
warnings.filterwarnings('ignore')

In [2]:
# Read in data for test.
features = pd.read_csv('data/pokemon.csv')
combats = pd.read_csv('data/combats.csv')
test_data = pd.read_csv('data/tests.csv')

In [3]:
# Explore data.
features[0:3]

,#,Name,Type 1,Type 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,80,82,83,100,100,80,1,False


In [4]:
column_name = list(features.columns + '_1') + list(features.columns + '_2') + ['win']
results = pd.DataFrame(columns=column_name)

In [5]:
progress = ProgressBar()
for num in progress(xrange(combats.shape[0]/2)):
    # Fetch the features of each player.
    player_1 = features[features['#']==combats['First_pokemon'].loc[num]]
    player_2 = features[features['#'] == combats['Second_pokemon'].loc[num]]
    # Fetch result of combat.
    if combats['First_pokemon'].loc[num] == combats['Winner'].loc[num]:
        win = [1]
    else:
        win = [0]
    # Assign vector combination to result dataframe.
    results.loc[num] = np.array(player_1).tolist()[0] + np.array(player_2).tolist()[0] + list(win)

100% |########################################################################|


In [6]:
results[0:3]

,#_1,Name_1,Type 1_1,Type 2_1,HP_1,Attack_1,Defense_1,Sp. Atk_1,Sp. Def_1,Speed_1,...,Type 2_2,HP_2,Attack_2,Defense_2,Sp. Atk_2,Sp. Def_2,Speed_2,Generation_2,Legendary_2,win
0,266.0,Larvitar,Rock,Ground,50.0,64.0,50.0,45.0,50.0,41.0,...,Dark,70.0,70.0,40.0,60.0,40.0,60.0,3.0,False,0.0
1,702.0,Virizion,Grass,Fighting,91.0,90.0,72.0,90.0,129.0,108.0,...,Fighting,91.0,129.0,90.0,72.0,90.0,108.0,5.0,True,0.0
2,191.0,Togetic,Fairy,Flying,55.0,40.0,85.0,80.0,105.0,40.0,...,NaN,75.0,75.0,75.0,125.0,95.0,40.0,5.0,False,0.0


In [7]:
# Explore data.
combats[0:3]

,First_pokemon,Second_pokemon,Winner
0,266,298,298
1,702,701,701
2,191,668,668


In [8]:
# Explore data.
results[0:3]

,#_1,Name_1,Type 1_1,Type 2_1,HP_1,Attack_1,Defense_1,Sp. Atk_1,Sp. Def_1,Speed_1,...,Type 2_2,HP_2,Attack_2,Defense_2,Sp. Atk_2,Sp. Def_2,Speed_2,Generation_2,Legendary_2,win
0,266.0,Larvitar,Rock,Ground,50.0,64.0,50.0,45.0,50.0,41.0,...,Dark,70.0,70.0,40.0,60.0,40.0,60.0,3.0,False,0.0
1,702.0,Virizion,Grass,Fighting,91.0,90.0,72.0,90.0,129.0,108.0,...,Fighting,91.0,129.0,90.0,72.0,90.0,108.0,5.0,True,0.0
2,191.0,Togetic,Fairy,Flying,55.0,40.0,85.0,80.0,105.0,40.0,...,NaN,75.0,75.0,75.0,125.0,95.0,40.0,5.0,False,0.0


In [9]:
reload(data_clean)
reload(feature_engineering)

<module 'feature_engineering' from 'feature_engineering.pyc'>

In [100]:
target_column = 'win'
id_columns = ['#_1', 'Name_1', '#_2', 'Name_2']
result_1 = data_clean.wash_data(results, target_column, id_columns, outlier=False)

The ratio of outliers is 0.008252
The row index of outliers is as follows: [1, 25, 51, 260, 599, 677, 731, 776, 788, 862, 913, 1003, 1123, 1203, 1468, 1550, 1783, 1794, 1817, 1865, 1992, 2038, 2398, 2593, 2916, 2970, 2973, 3351, 3546, 3570, 3765, 3895, 3961, 4133, 4289, 4345, 4547, 4578, 4634, 4724, 4788, 4801, 4829, 4846, 4854, 5013, 5231, 5243, 5491, 5529, 5566, 5751, 5757, 6139, 6216, 6293, 6388, 6434, 6445, 6546, 6665, 6819, 7039, 7110, 7198, 7457, 7573, 7601, 7667, 7723, 7744, 7955, 8114, 8223, 8308, 8329, 8455, 8616, 8669, 8792, 8795, 9211, 9250, 9385, 9592, 9594, 9752, 9817, 9832, 9852, 10158, 10303, 10326, 10342, 10350, 10386, 10532, 10555, 10868, 11125, 11203, 11324, 11401, 11419, 11423, 11682, 11712, 11759, 11765, 12228, 12288, 12474, 12479, 12697, 12951, 13012, 13088, 13165, 13211, 13286, 13506, 13524, 13577, 13709, 13943, 14349, 14591, 14599, 14690, 14722, 14792, 14894, 14945, 14995, 15398, 15401, 15626, 15782, 15982, 16033, 16098, 16295, 16628, 16904, 17007, 17164, 17178, 

In [134]:
result_1[0:3]

,Type 1_1,Type 2_1,HP_1,Attack_1,Defense_1,Sp. Atk_1,Sp. Def_1,Speed_1,Generation_1,Legendary_1,...,Type 2_2,HP_2,Attack_2,Defense_2,Sp. Atk_2,Sp. Def_2,Speed_2,Generation_2,Legendary_2,win
0,Rock,Ground,50.0,64.0,50.0,45.0,50.0,41.0,2.0,False,...,Dark,70.0,70.0,40.0,60.0,40.0,60.0,3.0,False,0.0
1,Grass,Fighting,91.0,90.0,72.0,90.0,129.0,108.0,5.0,True,...,Fighting,91.0,129.0,90.0,72.0,90.0,108.0,5.0,True,0.0
2,Fairy,Flying,55.0,40.0,85.0,80.0,105.0,40.0,2.0,False,...,NaN,75.0,75.0,75.0,125.0,95.0,40.0,5.0,False,0.0


In [138]:
result_2 = data_clean.sample_data(result_1, target_column, id_columns, method='under-sampling')

In [136]:
result_2[0:3]

,Type 1_1,Type 2_1,HP_1,Attack_1,Defense_1,Sp. Atk_1,Sp. Def_1,Speed_1,Generation_1,Legendary_1,...,Type 2_2,HP_2,Attack_2,Defense_2,Sp. Atk_2,Sp. Def_2,Speed_2,Generation_2,Legendary_2,win
0,Psychic,missing,120.0,70.0,120.0,75.0,130.0,85.0,4.0,False,...,Psychic,80.0,80.0,90.0,110.0,130.0,110.0,3.0,True,0.0
1,Ice,Water,90.0,60.0,70.0,75.0,70.0,45.0,3.0,False,...,missing,100.0,75.0,115.0,90.0,115.0,85.0,2.0,True,0.0
2,Electric,missing,60.0,85.0,49.0,60.0,49.0,60.0,4.0,False,...,missing,75.0,100.0,110.0,45.0,55.0,65.0,1.0,False,0.0


In [135]:
num_columns, cate_columns = feature_engineering.column_type_detection(result_2, id_columns, target_column)

In [139]:
cate_fe = feature_engineering.CategoryFeatureEngineer(cate_columns)
result_3 = cate_fe.fit_transform(result_2)

In [140]:
result_3.shape

(21267, 95)

In [141]:
result_3[0:3]

,Type 1_1,Type 1_1_Bug,Type 1_1_Dark,Type 1_1_Dragon,Type 1_1_Electric,Type 1_1_Fairy,Type 1_1_Fighting,Type 1_1_Fire,Type 1_1_Ghost,Type 1_1_Grass,...,Speed_1,Generation_1,HP_2,Attack_2,Defense_2,Sp. Atk_2,Sp. Def_2,Speed_2,Generation_2,win
0,14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,85.0,4.0,80.0,80.0,90.0,110.0,130.0,110.0,3.0,0.0
1,10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,45.0,3.0,100.0,75.0,115.0,90.0,115.0,85.0,2.0,0.0
2,3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,60.0,4.0,75.0,100.0,110.0,45.0,55.0,65.0,1.0,0.0


In [112]:
outlier_detector = data_clean.OutlierDetector(result_3, target_column, id_columns)
outlier_index = outlier_detector.isolation_forest()

The ratio of outliers is 0.009075


In [146]:
reload(data_clean)
reload(feature_engineering)
import util
reload(util)
num_fe = feature_engineering.NumericFeatureEngineer(num_columns)
result_4 = num_fe.fit_transform(result_3)

HP_1
('HP_1', ': Statistics=0.923, p=0.000') Sample does not look Gaussian (reject H0) 
There is no outlier.
Attack_1
('Attack_1', ': Statistics=0.978, p=0.000') Sample does not look Gaussian (reject H0) 
There is no outlier.
Defense_1
('Defense_1', ': Statistics=0.939, p=0.000') Sample does not look Gaussian (reject H0) 
There is no outlier.
Sp. Atk_1
('Sp. Atk_1', ': Statistics=0.960, p=0.000') Sample does not look Gaussian (reject H0) 
There is no outlier.
Sp. Def_1
('Sp. Def_1', ': Statistics=0.965, p=0.000') Sample does not look Gaussian (reject H0) 
There is no outlier.
Speed_1
('Speed_1', ': Statistics=0.986, p=0.000') Sample does not look Gaussian (reject H0) 
There is no outlier.
Generation_1
('Generation_1', ': Statistics=0.906, p=0.000') Sample does not look Gaussian (reject H0) 
There is no outlier.
HP_2
('HP_2', ': Statistics=0.922, p=0.000') Sample does not look Gaussian (reject H0) 
There is no outlier.
Attack_2
('Attack_2', ': Statistics=0.978, p=0.000') Sample does not

In [147]:
result_4.shape

(21267, 109)

In [125]:
result_4[0:3]

,Type 1_1,Type 1_1_Bug,Type 1_1_Dark,Type 1_1_Dragon,Type 1_1_Electric,Type 1_1_Fairy,Type 1_1_Fighting,Type 1_1_Fire,Type 1_1_Ghost,Type 1_1_Grass,...,Speed_1_flag_flag,Generation_1_flag_flag,HP_2_flag_flag,Attack_2_flag_flag,Defense_2_flag_flag,Sp. Atk_2_flag_flag,Sp. Def_2_flag_flag,Speed_2_flag_flag,Generation_2_flag_flag,win_flag_flag
0,14.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [167]:
reload(data_clean)
reload(feature_engineering)
result_5 = data_clean.sample_data(result_3, target_column, id_columns, method='over-sampling')

In [168]:
result_5[0:3]

,Type 1_1,Type 1_1_Bug,Type 1_1_Dark,Type 1_1_Dragon,Type 1_1_Electric,Type 1_1_Fairy,Type 1_1_Fighting,Type 1_1_Fire,Type 1_1_Ghost,Type 1_1_Grass,...,Speed_1,Generation_1,HP_2,Attack_2,Defense_2,Sp. Atk_2,Sp. Def_2,Speed_2,Generation_2,win
0,14.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,85.0,4.0,80.0,80.0,90.0,110.0,130.0,110.0,3.0,0.0
1,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,45.0,3.0,100.0,75.0,115.0,90.0,115.0,85.0,2.0,0.0
2,3.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,60.0,4.0,75.0,100.0,110.0,45.0,55.0,65.0,1.0,0.0


In [153]:
target_converter = feature_engineering.TargetConverter(target_column)
result_6 = target_converter.fit_transform(result_5)

In [154]:
result_6[0:3]

,Type 1_1,Type 1_1_Bug,Type 1_1_Dark,Type 1_1_Dragon,Type 1_1_Electric,Type 1_1_Fairy,Type 1_1_Fighting,Type 1_1_Fire,Type 1_1_Ghost,Type 1_1_Grass,...,Speed_1_flag,Generation_1_flag,HP_2_flag,Attack_2_flag,Defense_2_flag,Sp. Atk_2_flag,Sp. Def_2_flag,Speed_2_flag,Generation_2_flag,win
0,14.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [71]:
reload(feature_filtering)

<module 'feature_filtering' from 'feature_filtering.py'>

In [155]:
feature_filter = feature_filtering.FeatureFilter(target_column)
result_7 = feature_filter.fit_transform(result_6)

In [156]:
result_7[0:3]

,Attack_1,Attack_2,Defense_1,Defense_2,Generation_1,Generation_2,HP_1,HP_2,Legendary_1,Legendary_1_False,...,Type 2_2_Ground,Type 2_2_Ice,Type 2_2_Others,Type 2_2_Poison,Type 2_2_Psychic,Type 2_2_Rock,Type 2_2_Steel,Type 2_2_Water,Type 2_2_missing,win
0,-0.227,0.159,1.390,0.623,0.465,-0.115,1.865,0.544,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,-0.558,0.006,-0.017,1.259,-0.109,-0.751,0.890,1.237,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.233,0.733,-0.817,1.140,0.465,-1.499,-0.303,0.356,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [171]:
reload(feature_filtering)
feature_embedded = feature_filtering.FeatureEmbedded(target_column)
result_8 = feature_embedded.fit_transform(result_5)

In [172]:
result_8[0:3]

,Attack_1,Attack_2,Defense_1,Defense_2,Generation_1,Generation_2,HP_1,HP_2,Legendary_1,Legendary_1_True,...,Type 2_1_Flying,Type 2_1_Ghost,Type 2_1_Ground,Type 2_1_missing,Type 2_2,Type 2_2_Flying,Type 2_2_Ghost,Type 2_2_Ground,Type 2_2_missing,win
0,70.0,80.0,120.0,90.0,4.0,3.0,120.0,80.0,0.0,0.0,...,0.0,0.0,0.0,1.0,12.0,0.0,0.0,0.0,0.0,0.0
1,60.0,75.0,70.0,115.0,3.0,2.0,90.0,100.0,0.0,0.0,...,0.0,0.0,0.0,0.0,16.0,0.0,0.0,0.0,1.0,0.0
2,85.0,100.0,49.0,110.0,4.0,1.0,60.0,75.0,0.0,0.0,...,0.0,0.0,0.0,1.0,16.0,0.0,0.0,0.0,1.0,0.0


In [163]:
reload(feature_filtering)
feature_decomposition = feature_filtering.FeatureDecomposition(target_column)
result_9 = feature_decomposition.fit_transform(result_8)

In [164]:
result_9[0:3]

,pca_0,pca_1,pca_2,pca_3,pca_4,pca_5,pca_6,pca_7,pca_8,pca_9,...,pca_28,pca_29,pca_30,pca_31,pca_32,pca_33,pca_34,pca_35,pca_36,win
0,-9.480829,0.408118,-1.786421,-1.902832,-2.729728,-2.242667,-0.913232,-0.534315,-0.041503,0.226336,...,-0.000464,0.408986,0.144877,0.116797,-0.529704,-0.039068,-0.011255,0.358365,-0.056351,0.0
1,4.639886,-5.685930,-0.260352,-6.487136,-0.337621,-2.236622,-0.377820,-0.303051,0.007611,0.743577,...,0.186139,-0.025950,-0.015258,0.108686,-0.065796,-0.087170,-0.099771,-0.028602,0.023689,0.0
2,2.414124,4.765439,-0.902438,-7.284129,0.442914,-0.544740,-0.171051,0.132065,-1.247249,1.254966,...,-0.158132,0.642693,-0.128571,-0.067796,0.314700,-0.033077,-0.145177,0.153596,0.072988,0.0


### Build regression.

In [173]:
# Build Logistic Regression model to test improvement of accuracy after applying feature transformation.
# The task is to predict whether it's legendary or not.
logreg = linear_model.LogisticRegression(C=1e5)
X = result_8[result_8.columns.difference([target_column])]  
Y = result_8.win
log_re = logreg.fit(X, Y)

In [109]:
# The performance before numerical feature transformation.
# result_3.
logreg.score(X, Y) 

0.885127192363756

In [145]:
# result_4.
logreg.score(X, Y) 

0.8828701744486763

In [152]:
# result_5.
logreg.score(X, Y) 

0.885695652173913

In [170]:
# result_5X.
logreg.score(X, Y) 

0.8885652173913043

In [158]:
# result_7.
logreg.score(X, Y) 

0.885695652173913

In [162]:
# result_8.
logreg.score(X, Y) 

0.8868260869565218

In [174]:
# result_8X.
logreg.score(X, Y) 

0.8878260869565218

In [166]:
# result_9.
logreg.score(X, Y) 

0.8842173913043478

In [178]:
from step_by_step import FeatureEngineering
reload(step_by_step)

<module 'step_by_step' from 'step_by_step.py'>

In [179]:
feature_engineering = step_by_step.FeatureEngineering(id_columns, target_column)
total_result = feature_engineering.fit_transform(results)

The ratio of outliers is 0.008252
The row index of outliers is as follows: [1, 25, 51, 260, 599, 677, 731, 776, 788, 862, 913, 1003, 1123, 1203, 1468, 1550, 1783, 1794, 1817, 1865, 1992, 2038, 2398, 2593, 2916, 2970, 2973, 3351, 3546, 3570, 3765, 3895, 3961, 4133, 4289, 4345, 4547, 4578, 4634, 4724, 4788, 4801, 4829, 4846, 4854, 5013, 5231, 5243, 5491, 5529, 5566, 5751, 5757, 6139, 6216, 6293, 6388, 6434, 6445, 6546, 6665, 6819, 7039, 7110, 7198, 7457, 7573, 7601, 7667, 7723, 7744, 7955, 8114, 8223, 8308, 8329, 8455, 8616, 8669, 8792, 8795, 9211, 9250, 9385, 9592, 9594, 9752, 9817, 9832, 9852, 10158, 10303, 10326, 10342, 10350, 10386, 10532, 10555, 10868, 11125, 11203, 11324, 11401, 11419, 11423, 11682, 11712, 11759, 11765, 12228, 12288, 12474, 12479, 12697, 12951, 13012, 13088, 13165, 13211, 13286, 13506, 13524, 13577, 13709, 13943, 14349, 14591, 14599, 14690, 14722, 14792, 14894, 14945, 14995, 15398, 15401, 15626, 15782, 15982, 16033, 16098, 16295, 16628, 16904, 17007, 17164, 17178, 

In [180]:
total_result[0:3]

,pca_0,pca_1,pca_2,pca_3,pca_4,pca_5,pca_6,pca_7,pca_8,pca_9,...,pca_28,pca_29,pca_30,pca_31,pca_32,pca_33,pca_34,pca_35,pca_36,win
0,-9.480829,0.408118,-1.786421,-1.902832,-2.729728,-2.242667,-0.913232,-0.534315,-0.041503,0.226336,...,-0.000464,0.408986,0.144877,0.116797,-0.529704,-0.039068,-0.011255,0.358365,-0.056351,0.0
1,4.639886,-5.685930,-0.260352,-6.487136,-0.337621,-2.236622,-0.377820,-0.303051,0.007611,0.743577,...,0.186139,-0.025950,-0.015258,0.108686,-0.065796,-0.087170,-0.099771,-0.028602,0.023689,0.0
2,2.414124,4.765439,-0.902438,-7.284129,0.442914,-0.544740,-0.171051,0.132065,-1.247249,1.254966,...,-0.158132,0.642693,-0.128571,-0.067796,0.314700,-0.033077,-0.145177,0.153596,0.072988,0.0
